In [12]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [13]:
quarterly = pd.read_sql(
    """
    select datekey, reportperiod, ticker, netinc, equity
    from sf1
    where dimension='ARQ' and equity>0
    order by ticker, datekey
    """,
    conn
)
quarterly = quarterly.dropna()

In [14]:
quarterly["equitylag"] = quarterly.groupby("ticker").equity.shift()
quarterly["roe"] = quarterly.netinc / quarterly.equitylag

# save last report for each firm
quarterly = quarterly.groupby("ticker").last()
quarterly = quarterly[quarterly.reportperiod.astype(str)>="2022-06-01"]

# drop variables other than roe and ticker (ticker=index)
quarterly = quarterly[["roe"]]

In [15]:
daily1 = pd.read_sql(
    """ 
    select date, ticker, pb, marketcap
    from daily 
    where date>='2023-01-01' and pb>0
    order by ticker, date
    """,
    conn
)
daily1 = daily1.dropna()
daily1 = daily1.groupby("ticker").last()

In [16]:
daily2 = pd.read_sql(
    """ 
    select date, ticker, close_ as price
    from sep 
    where date>='2023-01-01'
    order by ticker, date
    """,
    conn
)
daily2 = daily2.dropna()
daily2 = daily2.groupby("ticker").last()

In [17]:
df = pd.concat((quarterly, daily1, daily2), axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3606 entries, A to YTPG
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   roe        3352 non-null   float64
 1   date       3283 non-null   object 
 2   pb         3283 non-null   float64
 3   marketcap  3283 non-null   float64
 4   date       3541 non-null   object 
 5   price      3541 non-null   float64
dtypes: float64(4), object(2)
memory usage: 197.2+ KB


In [18]:
df = df[df.price>5]
df.shape

(2579, 6)

In [19]:
df["size_rank"] = df.marketcap.rank(ascending=False)
df = df[df.size_rank > 500]

In [29]:
filter1 = df.roe >= df.roe.quantile(0.7)
filter2 = df.pb <= df.pb.quantile(0.3)
df = df[filter1 & filter2].copy()
ticks = df.index.to_list()
len(ticks)

109

In [25]:
!pip install --upgrade alpaca-py

  Using cached alpaca_py-0.7.0-py3-none-any.whl (95 kB)
  Using cached msgpack-1.0.4-cp310-cp310-win_amd64.whl (61 kB)
  Using cached websockets-10.4-cp310-cp310-win_amd64.whl (101 kB)
  Using cached pydantic-1.10.4-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached sseclient_py-1.7.2-py2.py3-none-any.whl (8.4 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
from alpaca.data import StockHistoricalDataClient
from alpaca.trading.client import TradingClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

KEY = "your_key"
SECRET_KEY = "your_secret_key"

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

In [30]:
params = StockLatestQuoteRequest(symbol_or_symbols=ticks)
quotes = data_client.get_stock_latest_quote(params)

# convert to dataframe

sers = [pd.Series(dict(x)) for x in quotes.values()]
df = pd.concat(sers, axis=1)
df.columns = [x for x in quotes]
df = df.T
df = df.drop(columns=["symbol"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109 entries, TOL to SCHL
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   timestamp     109 non-null    datetime64[ns, UTC]
 1   ask_exchange  109 non-null    object             
 2   ask_price     109 non-null    object             
 3   ask_size      109 non-null    object             
 4   bid_exchange  109 non-null    object             
 5   bid_price     109 non-null    object             
 6   bid_size      109 non-null    object             
 7   conditions    109 non-null    object             
 8   tape          109 non-null    object             
dtypes: datetime64[ns, UTC](1), object(8)
memory usage: 8.5+ KB


In [32]:
df["trade"] = df.ask_price.apply(lambda p: int(1000/p) if p !=0 else 0)
ticks = df[df.trade>0].index.to_list()

In [ ]:
for tick in ticks: 
    market_order_data = MarketOrderRequest(
        symbol=tick,
        qty=df.trade.loc[tick],
        side=OrderSide.BUY,
        time_in_force=TimeInForce.DAY
    )
    market_order = trading_client.submit_order(
        order_data=market_order_data
    )